In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
%cd ../

/home/offendo/Documents/masters/nlp243/best-words/src


<IPython.core.display.Javascript object>

# General setup
- imports
- reading in training data
- setting up `device` if cuda is available

In [3]:
# Local stuff
import data
from nli.selector import Selector, Embedder
import nli.lstm as lstm
import nli.mlp as mlp
from nli.trainer import FastTrainer

# Utilities
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Pytorch
import torch

torch.multiprocessing.set_start_method("spawn", force=True)
from torch.utils.data import DataLoader
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

<IPython.core.display.Javascript object>

In [4]:
train = data.get_train("../data/train.jsonl")
# train = train.explode("evidence").reset_index()
train, test = train_test_split(train)

<IPython.core.display.Javascript object>

# Sentence selection

In [5]:
torch.cuda.empty_cache()
em = Embedder()
em.model = em.model.to(device)
sel = Selector(em)

<IPython.core.display.Javascript object>

In [6]:
train_dataset = data.FastDataset(train)
test_dataset = data.TestDataset(test)

<IPython.core.display.Javascript object>

In [7]:
NUM_SENTS = 5
RETRIEVER = data.OracleDocRetriever("../data/wiki.db")
SELECTOR = sel


def prepare(batch):
    return data.collate(
        batch,
        NUM_SENTS,
        RETRIEVER,
        SELECTOR,
        oracle_doc_ret=True,
    )


train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=prepare,
    num_workers=4,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=prepare,
    num_workers=4,  # doesn't work with more than 1 and a sqlite connection
)

<IPython.core.display.Javascript object>

In [8]:
# Model params
EMBEDDING_DIM = em.model.get_sentence_embedding_dimension()
HIDDEN_DIM = 100
OUTPUT_DIM = 3  # refute, not enough info, support
N_LAYERS = 2
DROPOUT = 1e-1
BIDIRECTIONAL = True
# Loss fn params
WEIGHT_DECAY = 1e-4
N_EPOCHS = 3
LR = 1e-3
LR_DECAY = 1e-3

<IPython.core.display.Javascript object>

In [9]:
model = lstm.LSTMClassifier(
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
    n_layers=N_LAYERS,
    dropout=DROPOUT,
    bidirectional=BIDIRECTIONAL,
    pad_idx=train_dataset.input_pad_idx,
)
model.to(device)
state_dict = torch.load("../models/bilstm-nli-model-2.pt")
model.load_state_dict(state_dict)
optimizer = optim.Adam(model.parameters(), weight_decay=WEIGHT_DECAY, lr=LR)
loss_fn = torch.nn.CrossEntropyLoss(
    ignore_index=train_dataset.output_pad_idx,
    reduction="sum",
)
model.share_memory()

<IPython.core.display.Javascript object>

In [10]:
trainer = FastTrainer(model, optimizer, loss_fn, device, log_every_n=1)
labels = {0: "REFUTES", 1: "NOT ENOUGH INFO", 2: "SUPPORT"}

<IPython.core.display.Javascript object>

In [11]:
small_test_dataset = data.TestDataset(test[:1000])
small_test_loader = DataLoader(
    small_test_dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=prepare,
    num_workers=0,  # doesn't work with more than 1 and a sqlite connection
)
trainer.evaluate(small_test_loader, labels)

  0%|          | 0/12 [00:00<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 246754) exited unexpectedly

<IPython.core.display.Javascript object>